# RSV e Influenza

## Librerías

In [2]:
import pandas as pd
import numpy as np
from scipy.integrate import odeint
import random
import matplotlib.pyplot as plt

## Cargamos los datos

In [3]:
# Cargar datos de influenza y RSV
influenza_data = pd.read_excel('influenza.xlsx')
rsv_data = pd.read_csv('rsv20032009.csv')

# Convertir los datos a arrays de NumPy
influenza_data = np.array(influenza_data['Influenza'])
rsv_data = np.array(rsv_data)

c:\Users\lebue\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


## Modelo SEIR

In [4]:
# Definir el modelo SEIRS
def seirs_model(y, t, beta1, beta2, gamma1, gamma2, eta1, eta2, mu, theta1, theta2, sigma, N):
    S, E1, I1, R1, E2, I2, R2 = y
    
    # Ecuaciones del modelo SEIRS
    dSdt = mu * N - (beta1 * S * I1 / N + beta2 * S * I2 / N) - mu * S + theta1 * R1 + theta2 * R2
    dE1dt = (beta1 * S * I1 / N) - (mu + gamma1) * E1 - sigma * beta2 * E1 * I2 / N
    dI1dt = gamma1 * E1 - (mu + eta1) * I1 - sigma * beta2 * I1 * I2 / N
    dR1dt = eta1 * I1 - (mu + theta1) * R1 - sigma * beta2 * R1 * I2 / N
    dE2dt = (beta2 * S * I2 / N) + sigma * beta2 * (E1 + I1 + R1) * I2 / N - (mu + gamma2) * E2
    dI2dt = gamma2 * E2 - (mu + eta2) * I2
    dR2dt = eta2 * I2 - (mu + theta2) * R2
    
    return [dSdt, dE1dt, dI1dt, dR1dt, dE2dt, dI2dt, dR2dt]

# Función para simular el modelo en el tiempo
def sim_seirs_time(params, S0, E10, I10, R10, E20, I20, R20, N, time):
    beta1, beta2, gamma1, gamma2, eta1, eta2, mu, theta1, theta2, sigma = params
    y0 = [S0, E10, I10, R10, E20, I20, R20]
    ret = odeint(seirs_model, y0, time, args=(beta1, beta2, gamma1, gamma2, eta1, eta2, mu, theta1, theta2, sigma, N))
    S, E1, I1, R1, E2, I2, R2 = ret.T
    return I1, I2  # Retornamos los infectados de influenza y RSV

## MCMC

In [5]:
# Función de verosimilitud
def likelihood(data1, data2, predicted_cases1, predicted_cases2):
    # Truncar los datos observados a 52 semanas
    data1 = data1[:52]
    data2 = data2[:52]
    
    # Truncar las predicciones a 364 días (52 semanas)
    predicted_cases1 = predicted_cases1[:364]
    predicted_cases2 = predicted_cases2[:364]
    
    # Agregar las predicciones diarias a semanales
    predicted_weekly1 = predicted_cases1.reshape(-1, 7).sum(axis=1)  # Sumar cada 7 días
    predicted_weekly2 = predicted_cases2.reshape(-1, 7).sum(axis=1)  # Sumar cada 7 días
    
    # Calcular la verosimilitud
    sigma1 = np.std(data1)
    sigma2 = np.std(data2)
    log_likelihood1 = -0.5 * np.sum((data1 - predicted_weekly1)**2 / sigma1**2)
    log_likelihood2 = -0.5 * np.sum((data2 - predicted_weekly2)**2 / sigma2**2)
    return log_likelihood1 + log_likelihood2

# MCMC
def mcmc(data1, data2, param_0, iter, time, S0, E10, I10, R10, E20, I20, R20, N):
    # Parámetros iniciales
    beta1, beta2, gamma1, gamma2, eta1, eta2, mu, theta1, theta2, sigma = param_0
    accepted_params = []
    current_likelihood = likelihood(data1, data2, *sim_seirs_time(param_0, S0, E10, I10, R10, E20, I20, R20, N, time))
    
    for i in range(iter):
        # Proponer nuevos parámetros
        new_params = [
            beta1 + random.uniform(-0.01, 0.01),
            beta2 + random.uniform(-0.01, 0.01),
            gamma1 + random.uniform(-0.01, 0.01),
            gamma2 + random.uniform(-0.01, 0.01),
            eta1 + random.uniform(-0.01, 0.01),
            eta2 + random.uniform(-0.01, 0.01),
            mu + random.uniform(-0.001, 0.001),
            theta1 + random.uniform(-0.01, 0.01),
            theta2 + random.uniform(-0.01, 0.01),
            sigma + random.uniform(-0.01, 0.01)
        ]
        
        # Simular el modelo con los nuevos parámetros
        new_predicted1, new_predicted2 = sim_seirs_time(new_params, S0, E10, I10, R10, E20, I20, R20, N, time)
        
        # Calcular la nueva verosimilitud
        new_likelihood = likelihood(data1, data2, new_predicted1, new_predicted2)
        
        # Probabilidad de aceptación
        accept_prob = np.exp(new_likelihood - current_likelihood)
        
        if accept_prob > random.uniform(0, 1):
            # Aceptar nuevos parámetros
            beta1, beta2, gamma1, gamma2, eta1, eta2, mu, theta1, theta2, sigma = new_params
            current_likelihood = new_likelihood
            accepted_params.append(new_params)
    
    return accepted_params

C:\Users\lebue\AppData\Local\Temp\ipykernel_37496\4067317090.py:51: RuntimeWarning: overflow encountered in exp
  accept_prob = np.exp(new_likelihood - current_likelihood)
c:\Users\lebue\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\integrate\_odepack_py.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [6]:
# Parámetros iniciales
param_0 = [0.5, 0.5, 0.2, 0.2, 0.1, 0.1, 0.01, 0.01, 0.01, 0.5]

# Condiciones iniciales
N = 1000000  # Población total
S0 = N - 1   # Susceptibles iniciales
E10, I10, R10 = 0, 1, 0  # Expuestos, infectados y recuperados iniciales de influenza
E20, I20, R20 = 0, 1, 0  # Expuestos, infectados y recuperados iniciales de RSV

# Tiempo de simulación (diario)
time = np.arange(0, 365, 1)  # 1 año de simulación

# Ejecutar MCMC
accepted_params = mcmc(influenza_data, rsv_data, param_0, iter=10000, time=time, S0=S0, E10=E10, I10=I10, R10=R10, E20=E20, I20=I20, R20=R20, N=N)

# Convertir los parámetros aceptados a un array de NumPy
accepted_params = np.array(accepted_params)

# Graficar las distribuciones posteriores de los parámetros
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
for i in range(10):
    plt.subplot(3, 4, i+1)
    plt.hist(accepted_params[:, i], bins=30, color='blue', alpha=0.7)
    plt.title(f'Parámetro {i+1}')
plt.tight_layout()
plt.show()